In [9]:
!pip install fastai==1.0.61
!pip install timm

## Library & Framework

This project using fastaiv1

In [10]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastprogress import progress_bar

from PIL import Image
import os
from glob import glob
import cv2
from tqdm import tqdm
import multiprocessing as mp
import pydicom
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import sklearn

from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import timm
import pandas as pd
import numpy as np


from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
import torch
from torchvision import transforms

Variable config as a hyper-parametters object

In [11]:
config = {}

config['project_path'] = '/kaggle/input/rsna-pneumonia-detection-challenge'
config['img_input'] = os.path.join(config['project_path'],'stage_2_train_images')
config['working'] = '/kaggle/working'


config['size'] = 256

config['batch_size'] = 16

config['num_workers'] = 8
config['num_classes'] = 2
config['epochs'] = 1
config['lr'] = 5e-4
config['sch_lr'] = 0.9

config['pool'] = 6500

config['nb_iterations'] = 5

config['num_gist'] = 200

config['initial_decay_rate'] = 0.2
config['decay_rate'] = 0.2
config['thresh'] = None

config['discrim_ratio'] = 0.2

config['muy'] = 0.5

config['ite_momentum'] = 5

config['thres_momentum'] = 0.8

config['valid_ratio'] = 0.1

In [12]:
pd.options.mode.chained_assignment = None  # default='warn'

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

## Utils functions

In [13]:
def prepare_working(config):
    if os.path.isdir(os.path.join(config['working'],'used')):
        shutil.rmtree(os.path.join(config['working'],'used'))
        os.mkdir(os.path.join(config['working'],'used'))
    else:
        os.mkdir(os.path.join(config['working'],'used'))
        
    if os.path.isdir(os.path.join(config['working'],'unused')):
        shutil.rmtree(os.path.join(config['working'],'unused'))
        os.mkdir(os.path.join(config['working'],'unused'))
    else:
        os.mkdir(os.path.join(config['working'],'unused'))

    if os.path.isdir(os.path.join(config['working'],'pool')):
        shutil.rmtree(os.path.join(config['working'],'pool'))
        os.mkdir(os.path.join(config['working'],'pool'))
    else:
        os.mkdir(os.path.join(config['working'],'pool'))

    if os.path.isdir(os.path.join(config['working'],'test')):
        shutil.rmtree(os.path.join(config['working'],'test'))
        os.mkdir(os.path.join(config['working'],'test'))
    else:
        os.mkdir(os.path.join(config['working'],'test'))
        
    print('Prepare Working directory Done')

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized


def convert_dcm_png(zip_info, fold_dcm = config['img_input'], img_size = config['size']):

    fname, dest = zip_info
    
    path_dcm = os.path.join(fold_dcm, fname + '.dcm')
    
    dicom = pydicom.read_file(path_dcm)
    data = dicom.pixel_array
    
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    
    data = (data * 255).astype(np.uint8)
    
    img_rs = image_resize(data,img_size)
    
    path_png = os.path.join(dest,fname + '.png')
    
    cv2.imwrite(path_png,img_rs)    
    

def load_checkpoint(filepath):
    
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['model_state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    model.eval()
    return model



class poolDataset(Dataset):
    def __init__(self, dir_path, df_pool, transforms=None):
        """
        param:
            dir_path: is path of images
            df_pool: is dataframe of pool            
            transform:
            
        return:
            
        """
        self.transforms = transforms

        self.dir_path = dir_path

        self.imgs = list(df_pool['fname'])
        
    def __getitem__(self, item):
        
        img_path = os.path.join(self.dir_path, self.imgs[item])
        img = Image.open(img_path).convert('RGB')
        
        if self.transforms is not None:
            img = self.transforms(img)
        return img,self.imgs[item]
    
    def __len__(self):
        return len(self.imgs)
    
def entropy_rank(pred):
    en = np.zeros(len(pred))

    for i in range(0, len(pred)):
        en[i] = sum(-pred[i] * np.log(pred[i]))

    return np.argsort(en), en


def uncertain_set(en, nb_annotations):
    return en[0:nb_annotations]


def certain_set(en, thresh, initial_decay_rate, decay_rate):
    # Threshold updating <-- review
    if thresh == None:
        thresh = max(en) - ((max(en) - min(en)) * initial_decay_rate)
    else:
        thresh = thresh + (max(en) - thresh) * decay_rate

    return np.where(en < thresh)[0], thresh


def predictions_max_class(predictions):
    return np.argmax(predictions,axis=1)


def pseudo_label_error(pseudo_samples, true_samples):
    aux = 0
    for i in range(0, len(pseudo_samples)):
        if (pseudo_samples[i] == true_samples[i]).sum() != len(true_samples[i]):
            aux += 1
    return aux / len(true_samples)

def learn_get_preds(model, device, data_loader):

    confidences = []
    indices = []
    
    model.eval()
    
    with torch.no_grad():
        for batch in progress_bar(data_loader):
        
            imgs, fnames = batch
            
            imgs = imgs.to(device)
            
            logits = model(imgs)
            
            probabilities = torch.sigmoid(logits)
            
            confidences.extend(probabilities.cpu().tolist())
            
            indices.extend(fnames)
            
    conf = np.asarray(confidences)
    ind = np.asarray(indices)
    return conf, ind

def multi_process_copy_img(df):
    
    pool = mp.Pool(8)
    
    fnames = list(df['patientId'])
    dest = list(df['path'])

    zip_info = list(zip(fnames,dest))

    for _ in tqdm(pool.imap_unordered(convert_dcm_png,zip_info),total=len(zip_info)):
        pass 
    
def feature_extractor(learn, dataloader, is_train = True):
    
#     backbone = nn.Sequential(*list(list(learn.model.module[0].children())[:-1]))
#     backbone = nn.Sequential(*list(list(learn.model.module.children())[0][0].children())[:-1])
    backbone = nn.Sequential(*list(list(learn.model.module.children())[0].children())[:-1])
    
    list_features = []
    list_categories = []
    
    list_fnames = []
    
    if is_train:
        for x, y in progress_bar(dataloader):
            x, y = x.cuda(), y.cuda()
            backbone = backbone.cuda()

            feature = backbone(x).detach().cpu()
            list_features.append(feature)
            list_categories.append(y.detach().cpu())

        array_features = np.concatenate(list_features,axis=0)
        array_categories = np.concatenate(list_categories,axis=0)
        
        return array_features, array_categories
    
    else:
        for x,fname in progress_bar(dataloader):
            x  = x.cuda()
            backbone = backbone.cuda()

            feature = backbone(x).detach().cpu()
            
            list_features.append(feature)
            list_fnames.append(fname)
            
        array_features = np.concatenate(list_features,axis=0)
        array_fnames = np.concatenate(list_fnames,axis=0)
        
        return array_features, array_fnames
    
def query_update(df, fnames, labels):
    
    ## Update df_concat & move image from pool --> used
    
    for fname, label in tqdm(zip(fnames, labels),total = len(fnames)):

        df.loc[df['fname'] == fname, 'target'] = label
        df.loc[df['fname'] == fname, 'is_valid'] = False
        df.loc[df['fname'] == fname, 'is_anno'] = True
        
        move_source = os.path.join(config['working'],'pool',fname)
        move_dest = os.path.join(config['working'],'used',fname)
        
        shutil.move(move_source,move_dest)
        
        df.loc[df['fname'] == fname, 'path'] = os.path.join(config['working'],'used')
    
    print('Pseudo-labeling {} instances: Done'.format(len(fnames)))
    
    return df

def generate_gist(uncertain_samples):
    
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    df_gist = pd.DataFrame([])

    df_gist['fname'] = uncertain_samples
    
    pool_transforms = transforms.Compose([
                        transforms.Resize(config['size'],config['size']),
                        transforms.ToTensor(),
                        transforms.Normalize(mean, std)
                        ])

    gist_ds = poolDataset(
                        os.path.join(config['working'],'pool'),
                        df_gist,
                        pool_transforms
                        )

    dls_gist = DataLoader(
                        gist_ds, 
                        batch_size=config['batch_size'], 
                        num_workers=config['num_workers']
                        )
    return dls_gist


def gist_clustering(feature_gist, fnames_gist, feature_train, categories_train, discrim_ratio=config['discrim_ratio']):
    
    pos_idx = np.where(categories_train == 1)[0]
    neg_idx = np.where(categories_train == 0)[0]

    pos_feature = feature_train[pos_idx]
    neg_feature = feature_train[neg_idx]

    
    all_vectors = np.concatenate([pos_feature, neg_feature])
    
    kmeans = KMeans(n_clusters = 2 ,random_state=0).fit(all_vectors)
    labels = kmeans.predict(all_vectors)
    
    centroids = kmeans.cluster_centers_
    
    gist_distances= []
    
    pos_centroid = np.mean(pos_feature,axis=0)
    
    neg_centroid = np.mean(neg_feature,axis=0)
    
    ## verify centroids
    
    verify_centroids = {}
    
    if np.linalg.norm(pos_centroid-centroids[0]) < np.linalg.norm(pos_centroid-centroids[1]):
        pos_centroid = centroids[0]
        neg_centroid = centroids[1]
    else:
        pos_centroid = centroids[1]
        neg_centroid = centroids[0]
    
    kmeans_preds_class = kmeans.predict(feature_gist)
    
    dist_between_centroids = np.linalg.norm(neg_centroid - pos_centroid)
        
    decision_boundary = (dist_between_centroids/2)*discrim_ratio
    
    kmeans_preds_score = []
    
    for g in tqdm(feature_gist,total = feature_gist.shape[0]):
        dist_neg = np.linalg.norm(g - neg_centroid)
        dist_pos = np.linalg.norm(g - pos_centroid)
        
#         kmeans_preds_score.append((dist_neg,dist_pos))
        kmeans_preds_score.append(np.linalg.norm(dist_neg-dist_pos))
    
    kmeans_preds_score = np.array(kmeans_preds_score)
    
    fnames_array = np.array(fnames_gist)
    
#     indices_db = np.where(np.min(kmeans_preds_score,axis=1) <= decision_boundary)[0]

    indices_db = np.where(kmeans_preds_score > decision_boundary)[0]
    
    preds_class_db = kmeans_preds_class[indices_db]
    
    preds_score_db = kmeans_preds_score[indices_db]
    
    fnames_db = fnames_array[indices_db]

    return fnames_db.tolist(), preds_class_db.tolist(), kmeans_preds_score

def moving_avg(last_conf , curr_conf, muy = config['muy']):
    return muy*last_conf + (1-muy)*curr_conf

def move_to_unused(df, fnames):
    
    for fname in tqdm(fnames ,total = len(fnames)):

        df.loc[df['fname'] == fname, 'is_used'] = False
        
        move_source = os.path.join(config['working'],'used',fname)
        move_dest = os.path.join(config['working'],'unused',fname)
        
        shutil.move(move_source,move_dest)
        
        df.loc[df['fname'] == fname, 'path'] = os.path.join(config['working'],'used')
    
    print('Remove {} low confidence instances: Done'.format(len(fnames)))
    
    return df

def online_active_learning(df, 
                learn, 
                iterations = config['ite_momentum'], 
                thres = config['thres_momentum']):

    learn.unfreeze()
    
    print('Current iteration train: {0} neg / {1} pos'.format(
            learn.data.train_ds.y.items.tolist().count(0),
            learn.data.train_ds.y.items.tolist().count(1))
         )
    
    for epoch in range(config['ite_momentum']):
        print('Momentum at epoch: {}'.format(str(epoch)))
        learn.fit_one_cycle(config['epochs'], slice(config['lr']), pct_start=config['sch_lr'])
        preds_train = learn.get_preds(learn.data.train_dl)
        curr_conf = preds_train[0]
        if epoch != 0:
            moment_conf = moving_avg(last_conf, curr_conf)
            last_conf = moment_conf   
        else:
            last_conf = curr_conf      
            
            
    last_conf = np.array(last_conf)
    fnames_train = np.array([item.split('/')[-1] for item in learn.data.train_ds.x.items.tolist()])
    indices_low_conf = np.where(np.max(last_conf, axis=1) <= thres)[0]
    fnames_low_conf = fnames_train[indices_low_conf].tolist()
    df = move_to_unused(df, fnames_low_conf)
    dls_anno, dls_pool = create_dataloader(df)
    
    learn = Learner(data=dls_anno,                  
                    model=learn.model,
                    metrics=FBeta(beta=1))
    
    
    return df, learn

def gist_selection(X_gist_set, learn):
    
    dls_gist = generate_gist(X_gist_set)
            
    feature_train, categories_train = feature_extractor(learn, learn.data.train_dl.dl)

    feature_gist, fnames_gist = feature_extractor(learn, dls_gist, is_train=False)

    X_gist, y_gist, gist_sdb = gist_clustering(
                    feature_gist, 
                    fnames_gist, 
                    feature_train, 
                    categories_train)
    
    return X_gist, y_gist, gist_sdb

## Prepare Dataframe

In [14]:
def prepare_dataframe(config):
    
    df_ = pd.read_csv(os.path.join(config['project_path'],'stage_2_detailed_class_info.csv')).drop_duplicates(subset='patientId').reset_index(drop=True)
    
    df_['target'] = [1 if item == 'Lung Opacity' else 0 for item in list(df_['class'])]
    
    df_full, df_anno_valid = train_test_split(df_, test_size = config['valid_ratio'],random_state=0)

#     df_anno_full = df_full[(df_full['class']=='Normal')|(df_full['class']=='Lung Opacity')]
    
    ## fixed bug 
    
    df_anno_full, df_not_anno_full = train_test_split(df_full, test_size = config['anno_ratio'],random_state=0)
    
#     df_anno_full['target'] = [1 if item == 'Lung Opacity' else 0 for item in list(df_anno_full['class'])]
#     df_not_anno_full = df_full[df_full['class']=='No Lung Opacity / Not Normal']

    df_not_anno_full['target'] = 0

#     df_anno_used = df_anno_full.sample(config['used']).reset_index(drop=True)
    
    df_anno_used, _ = train_test_split(df_anno_full,test_size = (1 - config['used']/len(df_anno_full)))
    
    df_anno_train = df_anno_used.reset_index(drop=True)
    
#     df_anno_test = df_anno_full.sample(config['test']).reset_index(drop=True)

#     df_not_anno = df_not_anno_full.sample(config['pool']).reset_index(drop=True)
    df_not_anno = df_not_anno_full

#     df_anno_test = df_anno_test[~df_anno_test['patientId'].isin(df_anno_used['patientId'])]
#     df_anno_test['path'] = os.path.join(config['working'],'test')

#     df_anno_train, df_anno_valid = train_test_split(df_anno_used, test_size=0.2,random_state=0)

    df_anno_train['is_valid'] = False
    df_anno_train['is_used'] = True
    df_anno_train['is_anno'] = True
    df_anno_train['path'] = os.path.join(config['working'],'used')

    df_anno_valid['is_valid'] = True
    df_anno_valid['is_used'] = True
    df_anno_valid['is_anno'] =True
    df_anno_valid['path'] = os.path.join(config['working'],'used')

#     df_anno_test['is_valid'] = True
#     df_anno_test['is_used'] = True
#     df_anno_test['is_anno'] =True
#     df_anno_test['path'] = os.path.join(config['working'],'test')

    df_not_anno['is_valid'] = False
    df_not_anno['is_used'] = True
    df_not_anno['is_anno'] = False
    df_not_anno['path'] = os.path.join(config['working'],'pool')

    df_concat = pd.concat([df_anno_train,df_anno_valid,df_not_anno]).reset_index(drop=True)
    
    df_concat['fname'] = [item + '.png' for item in list(df_concat['patientId'])]
    
#     df_anno_test['fname'] = [item + '.png' for item in list(df_anno_test['patientId'])]


    ## copy image from Input to Working
    
    multi_process_copy_img(df_concat)
    
#     zip_concat = list(zip(list(df_concat['patientId']),list(df_concat['path'])))
#     for item in tqdm(zip_concat):
#         convert_dcm_png(item)

#     zip_test = list(zip(list(df_anno_test['patientId']),list(df_anno_test['path'])))
    
#     for item in tqdm(zip_test):
#         convert_dcm_png(item)
    
    print('Preparing Dataframe Done')
    
#     return df_concat, df_anno_test
    return df_concat

## Transform Augmentation & Create Dataloader

In [15]:
# def create_dataloader(df_concat, df_anno_test):
def create_dataloader(df_concat):

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    pool_transforms = transforms.Compose([
                            transforms.Resize(config['size'],config['size']),
                            transforms.ToTensor(),
                            transforms.Normalize(mean, std)
                            ])

    ex_p = 0.1

    extra = [*rand_resize_crop(
                config['size'], 
                max_scale=1.1),
             squish(
                scale=(0.975, 1.2), 
                p=ex_p),
             tilt(
                direction=(0, 3), 
                magnitude=(-0.3, 0.3), 
                p=ex_p),
             cutout(
                n_holes=(1, 10), 
                length=(3, 10), 
                p=ex_p*2)
            ]

    anno_transforms = get_transforms(
                max_rotate=15, 
                max_zoom=1.1, 
                max_lighting=0.2, 
                max_warp=0.15, 
                p_affine=0.5, 
                p_lighting=0.4, 
                xtra_tfms=extra)

    anno_transforms = list(anno_transforms)
    
    anno_transforms[1] = []
    
    dls_anno = (
                ImageList.from_df(
                    df=df_concat[
                        (df_concat['is_anno']==True)&(df_concat['is_used']==True)
                                ].reset_index(drop=True), 
                    path=os.path.join(config['working'],'used'), 
                    cols='fname')
                .split_from_df(col='is_valid')
                .label_from_df(cols=['target'], label_cls=CategoryList)
                .transform(anno_transforms, size=config['size'])
                .databunch(bs=config['batch_size'], num_workers=config['num_workers'])
                .normalize(imagenet_stats)
                )
    
#     df_pool_uniform = df_concat[df_concat['is_anno']==False].sample(config['pool']).reset_index(drop=True)

    df_pool = df_concat[df_concat['is_anno']==False]
    
    df_pool_uniform, _ = train_test_split(df_pool,test_size = (1 - config['pool']/len(df_pool)))
    
    df_pool_uniform = df_pool_uniform.reset_index(drop=True)

    pool_ds = poolDataset(
                    os.path.join(config['working'],'pool'),
                    df_pool_uniform,
                    pool_transforms
                    )

    dls_pool = DataLoader(
                    pool_ds, 
                    batch_size=config['batch_size'], 
                    num_workers=config['num_workers']
                    )
    
    print('Train {} samples'.format(str(dls_anno.train_ds.x.items.shape[0])))
    print('Valid {} samples'.format(str(dls_anno.valid_ds.x.items.shape[0])))
    print('Pool {} samples'.format(str(len(dls_pool.dataset.imgs))))
    
    return dls_anno, dls_pool

In [16]:
for ratio in sorted(list(np.array(list(range(3,10)))*0.1),reverse=True):

    config['used'] = int((1-ratio)*24000)

    config['anno_ratio'] = ratio

    prepare_working(config)

    config['thresh'] = 0.2

    print('='*80)
    print('Experiments with {} samples annotation'.format(str(config['used'])))

#     df_concat, df_anno_test = prepare_dataframe(config)

    df_concat = prepare_dataframe(config)

    fnames, labels = [], [] 

    for ite in range(config['nb_iterations']):

        if len(df_concat[df_concat['is_anno']==False]) != 0:

            print('*'*10)
            print('Iteration: {}'.format(str(ite)))

            df_concat = query_update(df_concat, fnames, labels)

            ## each iterations samples 6500 instances from the pool
    
            dls_anno, dls_pool = create_dataloader(df_concat)

            ## create Learner with dls_anno & model
            ## training with momentum Online Active learning
            
            if ite == 0:
                model = nn.Sequential(timm.create_model('resnext50_32x4d',pretrained=True,num_classes=config['num_classes']))
                learn = Learner(data=dls_anno,                  
                                model=model,
                                metrics=FBeta(beta=1))
                learn.model = nn.DataParallel(learn.model, device_ids=[0])
                
            else:
                learn = Learner(data=dls_anno,                  
                                model=learn.model,
                                metrics=FBeta(beta=1))
                
            df_concat, learn = online_active_learning(df_concat,learn)

            ## Query stratifies using pool dataloader

            confidence_scores, X_unlabeled = learn_get_preds(
                                model=learn.model,
                                device=device, 
                                data_loader=dls_pool
                              )

            predictions_rank, entropies = entropy_rank(confidence_scores)

            print('Entropy threshold for sampling: {}'.format(str(config['thresh'])))

            idx_certain_samples, config['thresh'] = certain_set(entropies, 
                            config['thresh'], 
                            config['initial_decay_rate'], 
                            config['decay_rate'])

            print('Certain {} instances'.format(str(idx_certain_samples.shape[0])))
            
            if idx_certain_samples.shape[0] != 0:
                ## Sampling high confidence score as certain instances

                X_certain = X_unlabeled[idx_certain_samples].tolist()

                X_gist, y_gist, gist_sdb = gist_selection(X_certain, learn)

                fnames = X_gist
                labels = y_gist

                print("Updating samples into Trainset: {0} neg / {1} pos".format(str(labels.count(0)),str(labels.count(1))))

                if labels.count(0) == labels.count(1) and labels.count(0) == 0:
                    break
            else:
                break
        else:
            break

Prepare Working directory Done
Experiments with 2399 samples annotation


100%|██████████| 26681/26681 [07:16<00:00, 61.19it/s]
0it [00:00, ?it/s]


Preparing Dataframe Done
**********
Iteration: 0
Pseudo-labeling 0 instances: Done
Train 2398 samples
Valid 2669 samples
Pool 6499 samples
Current iteration train: 1846 neg / 552 pos
Momentum at epoch: 0


epoch,train_loss,valid_loss,f_beta,time
0,0.444713,0.439960,0.469565,01:11


Momentum at epoch: 1


...

  0%|          | 6/1691 [00:00<00:31, 53.48it/s]

Updating samples into Trainset: 78 neg / 1613 pos
**********
Iteration: 3


100%|██████████| 1691/1691 [00:29<00:00, 56.90it/s]


Pseudo-labeling 1691 instances: Done


ValueError: test_size=-0.10469068660774994 should be either positive and smaller than the number of samples 5884 or a float in the (0, 1) range

# Investigate

# Evaluation

In [ ]:
feature_train, categories_train = feature_extractor(learn, learn.data.train_dl.dl)

pca = PCA(n_components=2)

feature_train_pca = pca.fit_transform(sklearn.preprocessing.normalize(feature_train))

x = [item[0] for item in feature_train_pca.tolist()]
y = [item[1] for item in feature_train_pca.tolist()]
color = ['darkorange' if item == 1 else 'navy' for item in categories_train.squeeze().tolist()]
plt.figure(figsize=(5, 5), dpi=80)
plt.scatter(x, y, color=color, alpha=0.5)
plt.show()

In [ ]:
# for x,y in dls_anno.valid_dl.dl:
#     print(x,y)
#     break

In [ ]:
feature_train, categories_train = feature_extractor(learn, dls_anno.valid_dl.dl)

pca = PCA(n_components=2)

feature_train_pca = pca.fit_transform(sklearn.preprocessing.normalize(feature_train))

x = [item[0] for item in feature_train_pca.tolist()]
y = [item[1] for item in feature_train_pca.tolist()]
color = ['darkorange' if item == 1 else 'navy' for item in categories_train.squeeze().tolist()]
plt.figure(figsize=(5, 5), dpi=80)
plt.scatter(x, y, color=color, alpha=0.5)
plt.show()

In [ ]:
# dls_anno

In [ ]:
# learn.get_preds()

In [ ]:
# test_predictions = []
# for test_image in progress_bar(dls_anno.test_dl.dl):
#     test_predictions.append(learn.predict(test_image)[0])

# print(classification_report(
#     y_true=list(df_anno_test['target']), 
#     y_pred=test_predictions))

In [ ]:
# feature_test = feature_extractor(learn, dls_pool, is_train=False)

In [ ]:
# train_diffs = []
# batch_size = config['batch_size']
# feature_train = torch.tensor(feature_train)
# train_diffs = torch.empty([feature_train.shape[0], feature_train.shape[0]])

# for i in tqdm(range((feature_train.shape[0]//batch_size) + 1)):
#     if i*batch_size > feature_train.shape[0]:
#         break
                      
#     end_idx = min(feature_train.shape[0], (i+1)*batch_size)
        
#     batch1 = feature_train[i*batch_size:end_idx]
    
#     batch2 = feature_train
#     diff = torch.norm(batch1[:, np.newaxis] - batch2, dim=-1)
    
#     train_diffs[i*batch_size:end_idx] = diff

In [ ]:
# test_diffs = []
# batch_size = config['batch_size']

# feature_test = torch.tensor(feature_test)

# test_diffs = torch.empty([feature_test.shape[0], feature_train.shape[0]])

# for i in tqdm(range((feature_test.shape[0]//batch_size) + 1)):
#     if i*batch_size > feature_test.shape[0]:
#         break
                      
#     end_idx = min(feature_test.shape[0], (i+1)*batch_size)
        
#     batch1 = feature_test[i*batch_size:end_idx]
    
#     batch2 = feature_train
#     diff = torch.norm(batch1[:, np.newaxis] - batch2, dim=-1)
    
#     test_diffs[i*batch_size:end_idx] = diff

In [ ]:
# test_diffs.shape

In [ ]:
# test_diffs = test_diffs.cpu().numpy()
# test_diffs.sort()

In [ ]:
# feature_train, categories_train = feature_extractor(learn, learn.data.valid_dl.dl)

# pca = PCA(n_components=2)

# feature_train_pca = pca.fit_transform(sklearn.preprocessing.normalize(feature_train))

# x = [item[0] for item in feature_train_pca.tolist()]
# y = [item[1] for item in feature_train_pca.tolist()]
# color = ['darkorange' if item == 1 else 'navy' for item in categories_train.squeeze().tolist()]

# plt.figure(figsize=(5, 5), dpi=80)

# plt.scatter(x, y, color=color, alpha=0.5)
# plt.show()

In [ ]:
# import plotly.figure_factory as ff
# #
# hist_data = [test_diffs[:, 1:12].mean(axis=-1), np.median(test_diffs[:, 1:12], axis=-1)]
# group_labels = ['mean', 'median']
# ff.create_distplot(hist_data, group_labels).show('iframe')

In [ ]:
# tmp = pd.DataFrame(test_diffs[:, :11])
# valid_index = tmp[tmp[0] < 3.15].index
# len(valid_index)